# Visual features extraction

In [1]:
from main_model import MainModel

In [ ]:
from config import cfg, input_shape
from utils import T, C, NUM_CLASSES
from detectron2.layers import ShapeSpec
import torch

B = 1
T = 5

input_shape = ShapeSpec(
    channels=C,
    height=256,
    width=256,
)

model = MainModel(
    input_shape_resnet=input_shape,
    resnet_cfg=cfg,
    T=T,
    num_classes=NUM_CLASSES,
)

visual_features = model.forward_visual_encoder(torch.randn(B, T, C, 256, 256))
audio_features = model.forward_audio_encoder(torch.randn(B, 4*T, 13))
fused_features = model.forward_fusion(visual_features, audio_features)
del visual_features, audio_features
head_output = model.forward_head(fused_features)
del fused_features

In [3]:
[head_output[i].shape for i in range(len(head_output))]

[torch.Size([5, 67, 64, 64]),
 torch.Size([5, 67, 32, 32]),
 torch.Size([5, 67, 16, 16]),
 torch.Size([5, 67, 8, 8]),
 torch.Size([5, 67, 4, 4])]

In [8]:
from detection_head import Head

In [9]:
detector = Head(nc=80, filters=[128, 128, 128, 128, 128])

In [14]:
fused_features = [
    feature_map.transpose(1, 2).view(B*T, 128, feature_map.shape[-2], feature_map.shape[-1])
    for feature_map in fused_features.values()
]

In [15]:
[fused_features[i].shape for i in range(len(fused_features))]

[torch.Size([5, 128, 64, 64]),
 torch.Size([5, 128, 32, 32]),
 torch.Size([5, 128, 16, 16]),
 torch.Size([5, 128, 8, 8]),
 torch.Size([5, 128, 4, 4])]

In [16]:
detector_out = detector(fused_features)

In [19]:
detector_out[1].shape

torch.Size([5, 144, 32, 32])